<a href="https://colab.research.google.com/github/danielka223/ds_workshop/blob/master/dsWorkshopAmazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="left">Helpfulness Prediction of Amazon Product Reviews</h1> 
<h2 align="left">DS Workshop - Fall 18/19 - Tel Aviv University</h2> 
<h3 align="left">By Daniel K.A, Ido Salomon, Itamar Mutzafi and Sagi Aharoni</h3> 




#Notebook Initialization
This section should only run once on a new runtime

In [0]:
#@title Import Python Libraries {display-mode: "form"}

import os
import pandas as pd
import json
import datetime

from google.colab import drive

from contextlib import contextmanager
from os.path import getsize, basename
from tqdm import tqdm

In [5]:
#@title Connect to Drive {display-mode: "form"}

drive.mount('/content/drive/')
            

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#@title Infrastracture Definitions

@contextmanager
def pbopen(filename):
    total = getsize(filename)
    pb = tqdm(total=total, unit="B", unit_scale=True,
              desc=basename(filename), miniters=1,
              ncols=80, ascii=True)

    def wrapped_line_iterator(fd):
        processed_bytes = 0
        for line in fd:
            processed_bytes += len(line)
            # update progress every MB.
            if processed_bytes >= 1024 * 1024:
                pb.update(processed_bytes)
                processed_bytes = 0

            yield line

        # finally
        pb.update(processed_bytes)
        pb.close()

    with open(filename) as fd:
        yield wrapped_line_iterator(fd)

# Data Collection


In [7]:
reviewsPath = "drive/Team Drives/DS Workshop/data/reviews_Office_Products_5.json"
with pbopen(reviewsPath) as file:
    reviews = pd.DataFrame(json.loads(line) for line in file)
###
reviews = reviews.rename(columns = {'asin':'PID'})
reviews['reviewTime'] = pd.to_datetime(reviews['reviewTime'], format = '%m %d, %Y')
reviews['thumbsUp'] = reviews.apply(lambda row: list(row['helpful'])[0], axis=1)
reviews['thumbsDown'] = reviews.apply(lambda row: list(row['helpful'])[1]-list(row['helpful'])[0], axis=1)

thumbs = pd.DataFrame(reviews.groupby('reviewerID').mean())
thumbs = thumbs[['thumbsUp','thumbsDown']]
thumbs.columns = ['thumbsUpAvg','thumbsDownAvg']
commentCount=pd.DataFrame(reviews.groupby('reviewerID').size())
commentCount.columns=['commentCount']
reviews = reviews.join(commentCount, on='reviewerID').join(thumbs, on='reviewerID')

reviews = reviews[['reviewerID','thumbsUp','thumbsDown','commentCount','thumbsUpAvg','thumbsDownAvg','overall','reviewText','reviewTime','PID','reviewerName','summary','unixReviewTime']]
reviews.head()

reviews_Office_Products_5.json: 100%|######| 55.9M/55.9M [00:00<00:00, 62.7MB/s]


,reviewerID,thumbsUp,thumbsDown,commentCount,thumbsUpAvg,thumbsDownAvg,overall,reviewText,reviewTime,PID,reviewerName,summary,unixReviewTime
0,A32T2H8150OJLU,3,1,27,7.37037,0.592593,5.0,"I bought my first HP12C in about 1984 or so, a...",2004-09-03,B00000JBLH,ARH,"A solid performer, and long time friend",1094169600
1,A3MAFS04ZABRGO,7,2,5,49.80000,3.400000,5.0,WHY THIS BELATED REVIEW? I feel very obliged t...,2007-12-15,B00000JBLH,"Let it Be ""Alan""","Price of GOLD is up, so don't bury the golden ...",1197676800
2,A1F1A0QQP2XVH5,3,0,30,2.30000,0.200000,2.0,I have an HP 48GX that has been kicking for mo...,2011-01-01,B00000JBLH,Mark B,"Good functionality, but not durable like old HPs",1293840000
3,A49R5DBXXQDE5,7,1,5,6.00000,0.400000,5.0,I've started doing more finance stuff recently...,2006-04-19,B00000JBLH,R. D Johnson,One of the last of an almost extinct species,1145404800
4,A2XRMQA6PJ5ZJ8,0,0,12,7.75000,0.500000,5.0,For simple calculations and discounted cash fl...,2013-08-04,B00000JBLH,Roger J. Buffington,Still the best,1375574400
